# Body of the Code

I feel that the following code is not very optimised. Still, it uses a bit of interesting maths and thought it was worth noting.

In [ ]:
import numpy as np

Below code takes input (M, N) array, X (e.g. output of the CUDA E-M OL integrator code), where M is the number of trajectories stored in X, and N is the number of integration steps taken when generating X. <br />
For the plotting, we require the corresponding dt, also specified when generating X.

In [ ]:
def acovs_WK_unbiased(X): # host function
  acovs = np.empty((np.shape(X)))
  for i in range(np.shape(X)[0]):
    acovs[i] = traj_acov_WK_unbiased(X[i])
  return acovs

def traj_acov_WK_unbiased(traj): # host function
  x = traj - np.mean(traj)
  n = len(x)
  npad = 1 << (2*n-1).bit_length() # zero-padding for fft
  # This line is cool but fucking unreadable, consider replacing with:
  #npad = 2 ** (2*n-1).bit_length() # or something similar, like
  #npad = 2 ** np.log(2*n-1)/np.log(2) # this is clearly inefficient lol
  Fx = np.fft.rfft(x, n=npad)
  ps = Fx * np.conjugate(Fx) # power spectrum - see Weiner Khinchin
  acov = np.fft.irfft(ps, n=npad)
  acov = acov[:n]
  count = np.arange(n, 0, -1) # counts for unbiased weighting; n, n-1, ..., 1
  return acov/count

Let's break apart the traj_acov_WK_unbiased() function. <br />
This function takes in a trajectory from X as an input. It is a (1,N) dimensional array. <br />
> x = traj - np.mean(traj) <br />

acov requires mean-zero data <br />
<br />

> n = len(x) <br />
> npad = 1 << (2*n-1).bit_length()

Zero-padding our length for efficiency.
3 things to unpack here: <br />
- n = len(x) is self-explanatory, it is the exact same as N (number of integration steps)
- NUMBER.bit_length() returns the number of bits required to represent a number in binary.
    - $4(b10) = 100(b2)$
    - $5(b10) = 101(b2)$
    - both require 3 bits to be represented - this function is basically just a more efficient ceil(log_2(NUMBER))
- L << R operator is a bitwise operator - tells you to shift bits of the number L by R bits;
    - $1 << 3 = 100(b2) = 4(b10)$
    - $5 << 4 = 1010000(b2) = 80(b10)$
    - $1 << k = 2^k(b10)$
- Why $(2n-1)$? Because we want to extend our trajectory array length to the next power of 2. Fast Fourier Transform (FFT) is the most efficient when operated on a length $2^k$ array for any integer $k$. <br />
<br />

> Fx = np.fft.rfft(x, n=npad)

Taking the Fourier transform (discrete) of our mean-corrected trajectory vector. This specific implementation is using the real fast Fourier transform (rfft) (no need to use fft which works with complex128 dtype for our implementation here). <br />
Now, why do we do this? This is the interesting part. <br />
<br />

> ps = Fx * np.conjugate(Fx)

By the Weiner-Khinchin theorem, we have that the Autocorrelation function [\*check this: need proper referencing; is it autocorrelation or autocovariance function?] is the Fourier transform of its power spectral density [\* check this: power spectral density definition?]. <br />
<br />

> acov = np.fft.irfft(ps, n=npad)

taking the inverse Fourier transform of the power spectrum <br />
<br />

> acov = acov[:n]

Remove zero-padding. <br />
<br />

> count = np.arange(n, 0, -1) <br />
> return acov/count

Here, count returns number of time signals used for computation of acovariance. This is required for the unbiased normalisation of the acovariance into autocorrelation!